In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "7"

In [2]:
import numpy as np
import pickle as pic

from sklearn.model_selection import train_test_split

In [3]:
import deep_learning

In [4]:
if True:
    with open("splitted_wavelets_haar_2.pic", "rb") as r:
        X_training, X_val, X_extraction = pic.load(r)
    with open("splitted_labels.pic", "rb") as r:
        y_training, y_val, y_extraction = pic.load(r)
else:
    with open("wavelets_haar_2.pic", "rb") as r:
        wavelets = pic.load(r)
    with open("labels.pic", "rb") as r:
        rws_perms_labels, round_perms_labels, copy_perms_labels, rws_masks_labels, round_masks_labels = pic.load(r)

    X_total, y_total = deep_learning.prepare_data_dl(wavelets, round_perms_labels, copy_perms_labels, round_masks_labels, rws_perms_labels, rws_masks_labels)

    profile, test = train_test_split(np.arange(X_total.shape[0]), train_size=500_000, random_state=0)

    X_profiling, X_extraction = X_total[profile], X_total[test]
    train, val = train_test_split(np.arange(X_profiling.shape[0]), test_size=0.1, random_state=0)
    X_training, X_val = X_profiling[train], X_profiling[val]

    with open("splitted_wavelets_haar_2.pic", "wb") as w:
        pic.dump((X_training, X_val, X_extraction), w)

    if False:
        y_profiling = {}
        y_training = {}
        y_val = {}
        y_extraction = {}
        for label in y_total.keys():
            print(label, end="\r")
            y_profiling[label], y_extraction[label] = y_total[label][profile], y_total[label][test]
            y_training[label], y_val[label] = y_profiling[label][train], y_profiling[label][val]

        with open("splitted_labels.pic", "wb") as w:
            pic.dump((y_training, y_val, y_extraction), w)
    else:
        with open("splitted_labels.pic", "rb") as r:
            y_training, y_val, y_extraction = pic.load(r)


In [5]:
train_gen = deep_learning.DataGenerator(X_training, y_training)
val_gen = deep_learning.DataGenerator(X_val, y_val)

In [6]:
resnet = deep_learning.ResNetSCA(epochs=1000, dataset_size=X_training.shape[0])
try:
    deep_learning.check_file_exists("./resnet_models/resnet_500000_haar_2.keras")
    from tensorflow.keras.models import load_model
    resnet.model = load_model("./resnet_models/resnet_500000_haar_2.keras")
except ValueError:
    pass

In [7]:
history = resnet.train_model_generator(train_gen, val_gen, "./resnet_models/resnet_500000_haar_2.keras", patience=50)

Epoch 1/1000
3516/3516 [==============================] - 5697s 2s/step - loss: 1100.9323 - rws_perm_output_loss: 0.0828 - rws_mask_0_0_output_loss: 2.7540 - rws_mask_0_1_output_loss: 2.7570 - rws_mask_1_0_output_loss: 2.7547 - rws_mask_1_1_output_loss: 2.7583 - rws_mask_2_0_output_loss: 2.7558 - rws_mask_2_1_output_loss: 2.7581 - rws_mask_3_0_output_loss: 2.7580 - rws_mask_3_1_output_loss: 2.7588 - rws_mask_4_0_output_loss: 2.7500 - rws_mask_4_1_output_loss: 2.7578 - rws_mask_5_0_output_loss: 2.7545 - rws_mask_5_1_output_loss: 2.7575 - rws_mask_6_0_output_loss: 2.7541 - rws_mask_6_1_output_loss: 2.7585 - rws_mask_7_0_output_loss: 2.7559 - rws_mask_7_1_output_loss: 2.7576 - rws_mask_8_0_output_loss: 2.7586 - rws_mask_8_1_output_loss: 2.7592 - rws_mask_9_0_output_loss: 2.7517 - rws_mask_9_1_output_loss: 2.7578 - rws_mask_10_0_output_loss: 2.7538 - rws_mask_10_1_output_loss: 2.7573 - rws_mask_11_0_output_loss: 2.7533 - rws_mask_11_1_output_loss: 2.7585 - rws_mask_12_0_output_loss: 2.7554

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical

y_predicted = load_model("./resnet_models/resnet_350000_52_epochs.keras").predict(X_extraction)
for i, k in enumerate(y_extraction.keys()):
    print(f"{k}: {np.count_nonzero(np.argmax(y_predicted[i], axis=1) == y_extraction[k])}/{y_extraction[k].shape[0]} = {np.count_nonzero(np.argmax(y_predicted[i], axis=1) == y_extraction[k]) / y_extraction[k].shape[0]}")

np.mean([categorical_crossentropy(to_categorical(y_extraction[k]), y_predicted[i]) for i, k in enumerate(y_extraction.keys())], axis=1)

1411/1411 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step
x_round: 24419/45133 = 0.5410453548401392
block_perm_0: 39134/45133 = 0.8670817362018922
mask_0_0: 354/45133 = 0.007843484811556953
mask_0_1: 365/45133 = 0.00808720891587087
mask_0_2: 305/45133 = 0.006757804710522234
mask_0_3: 302/45133 = 0.006691334500254802
mask_0_4: 332/45133 = 0.0073560366029291205
mask_0_5: 305/45133 = 0.006757804710522234
mask_0_6: 276/45133 = 0.006115259344603727


array([1.3536543, 0.3749589, 5.503599 , 5.5072737, 5.477    , 5.5458245,
       5.490437 , 5.568437 , 5.656007 ], dtype=float32)